
# 🧮 Pokémon Math Mystery — Kid‑Friendly Checkerboard Adventure

Welcome! This notebook is a calm, story‑first math game for practicing **multiplication** and **division**.
Move your token across the board by solving gentle puzzles. Each correct answer reveals a letter in a **mystery phrase**.
Reach the **GYM** to finish!

This notebook is organized in short steps. Every code cell is commented so it’s easy to follow.



## 1) Setup — Folders and Libraries

This cell:
- Creates an `assets/` folder (put images here, like `creature.png` or `token.webp`).
- Sets `ASSETS_DIR` to that folder.
- Imports the libraries we use.


In [ ]:

ASSETS_DIR = "assets"  # Change to "/content/assets" on Colab if you want.

import os
os.makedirs(ASSETS_DIR, exist_ok=True)
with open(os.path.join(ASSETS_DIR, "README.txt"), "w") as f:
    f.write("Put PNG/JPG/GIF/WEBP images here for your game token.\n")

# --- Imports used throughout the game ---
import math
import random
from typing import List, Tuple, Dict, Optional

import numpy as np
import matplotlib.pyplot as plt   # We do NOT set explicit colors or styles.
from matplotlib.patches import Rectangle

from IPython.display import display, HTML, Markdown
import ipywidgets as W
from pathlib import Path
from PIL import Image, ImageSequence, features  # Pillow handles images; WEBP if supported.

# Small helpers used in several places
def rand_item(xs):
    '''Return a random element from a non-empty list.'''
    return xs[random.randrange(len(xs))]

def clamp(x, lo, hi):
    '''Restrict x to the inclusive range [lo, hi].'''
    return max(lo, min(hi, x))



## 2) Story & Game Settings

- **Mystery phrases** — each correct answer unlocks one letter until the phrase is revealed.
- **Story beats** — short lines of story that appear as you move.
- **Setup controls** — kid‑friendly pickers for board size, math mode, factors, etc.


In [ ]:

# --- Story content: mystery phrases and flavor text ---
MYSTERY_PHRASES = [
    "TRAINER BADGE",
    "ELECTRIC ENERGY",
    "MYSTERY GYM",
    "POCKET CREATURES",
]

STORY_BEATS = [
    "A quiet morning in Pal Meadow. Your partner creature nudges you awake — the town's Gym is glowing with strange symbols.",
    "At the meadow gate, you find a note: 'Only those who understand equal groups may pass...'",
    "A breeze flips another card: 'Count your jumps on the number line to cross the river.'",
    "The Gym doors shimmer: 'Know a fact, and its family follows.'",
    "Inside the entry hall, a pedestal asks for a mystery phrase. Each correct step will reveal a letter...",
]

DEFAULT_FACTORS = [0, 1, 2, 5, 10]  # Matches early‑stage classroom sets.

# --- Token choices: emoji fallback + optional image filenames from ASSETS_DIR ---
EMOJI_TOKENS = ['⚡','🔥','💧','🍃','⭐','🌙','🪨','🌀']

def list_asset_images():
    '''List image files in ASSETS_DIR with common formats.'''
    p = Path(ASSETS_DIR)
    if not p.exists():
        return []
    return sorted([f.name for f in p.iterdir() if f.suffix.lower() in {'.png','.jpg','.jpeg','.gif','.webp'}])

# --- Game setup widgets (kid‑friendly tooltips) ---
size_dropdown = W.Dropdown(
    options=[6,7,8,9,10], value=8, description='Board',
    tooltip='How many squares across and down. Bigger board = longer game!'
)
mode_dropdown = W.Dropdown(
    options=[('Multiplication', 'multiply'), ('Division', 'divide'), ('Mixed', 'mixed')],
    value='mixed', description='Mode', tooltip='Pick the kind of math puzzles you\'ll solve.'
)
factors_text = W.Text(
    value='0,1,2,5,10', description='Factors',
    layout=W.Layout(width='300px'),
    tooltip='Numbers you\'ll multiply/divide by. Try adding 3,4,6,7,8,9 later!'
)
phrase_dropdown = W.Dropdown(
    options=MYSTERY_PHRASES, value=MYSTERY_PHRASES[0], description='Mystery',
    tooltip='What secret phrase will you reveal?'
)
show_numline = W.Checkbox(
    value=True, description='Show Number Line',
    tooltip='A helpful number line to see your jumps.'
)
show_groups  = W.Checkbox(
    value=True, description='Show Equal Groups',
    tooltip='Shows groups of things so you can \"see\" multiplication/division.'
)
show_factweb = W.Checkbox(
    value=True, description='Show Fact Web',
    tooltip='Shows how facts connect — 6×4=24, 4×6=24, 24÷6=4, 24÷4=6.'
)
token_type = W.Dropdown(
    options=['Emoji','Image'], value='Emoji', description='Token Type',
    tooltip='Use an emoji or pick a picture from the assets folder.'
)
token_emoji = W.Dropdown(
    options=EMOJI_TOKENS, value='⚡', description='Emoji',
    tooltip='Choose your token\'s emoji.'
)
token_image = W.Dropdown(
    options=(['(none)'] + list_asset_images()), value='(none)', description='Token Image',
    tooltip='Pick a picture from the assets folder.'
)

# --- Friendly intro shown above the controls ---
intro_text = '''
## Welcome to **Pokémon Math Mystery!** 🧮⚡

You're about to start an adventure where **math powers** unlock clues to a secret phrase.  
Each time you solve a problem, you'll earn a **letter** to reveal the mystery!

### Choose your game settings:
- **Board** – how big the board is (bigger = longer adventure).
- **Mode** – multiplication, division, or both.
- **Factors** – numbers used in your puzzles (start with 0,1,2,5,10).
- **Mystery** – the secret phrase you will uncover.
- **Hints** – Number Line, Equal Groups, and Fact Web.
- **Token** – an emoji or a picture for your game piece.
'''
display(Markdown(intro_text))

ui = W.VBox([
    W.HTML('<b>Game Setup</b>'),
    W.HBox([size_dropdown, mode_dropdown]),
    W.HBox([factors_text, phrase_dropdown]),
    W.HBox([show_numline, show_groups, show_factweb]),
    W.HBox([token_type, token_emoji, token_image]),
])
display(ui)



## 3) Image Loading (PNG/JPG/GIF/WEBP)

This helper safely opens images from the `assets/` folder.  
If the file is animated (GIF/WEBP), it uses the first frame. If loading fails, it returns `None` (the game falls back to emoji).


In [ ]:

WEBP_ENABLED = features.check('webp')

def load_token_image(path) -> Optional[Image.Image]:
    '''
    Open an image safely and return an RGBA Pillow Image, or None on error.
    Supports PNG/JPG/JPEG/GIF/WEBP. Animated images: use first frame.
    '''
    p = Path(path)
    if not p.exists():
        return None
    try:
        im = Image.open(p)
        if getattr(im, 'is_animated', False):
            im = next(ImageSequence.Iterator(im))
        return im.convert('RGBA')
    except Exception as e:
        if p.suffix.lower() == '.webp' and not WEBP_ENABLED:
            print('WEBP not supported by this Pillow build. Try: pip install --upgrade pillow')
        else:
            print(f'Image load error for {p.name}: {e}')
        return None



## 4) Game Logic — Problems, Answers, and Progress

- `Problem`: stores a single math prompt and the correct answer.
- `Game`: stores board size, current position, mystery phrase progress, etc.
- `make_problem`: creates a multiplication or division problem using chosen factors.


In [ ]:

class Problem:
    '''Container for a single math problem.'''
    def __init__(self, a, b, op, prompt, answer, dividend=None, divisor=None):
        self.a = a
        self.b = b
        self.op = op              # '×' or '÷'
        self.prompt = prompt      # text shown to the player
        self.answer = answer      # integer answer
        self.dividend = dividend  # for division
        self.divisor  = divisor   # for division

def parse_factors(text: str) -> List[int]:
    '''Parse a comma-separated list of integers 0..12. Fallback to DEFAULT_FACTORS if empty/invalid.'''
    xs = []
    for part in text.split(','):
        part = part.strip()
        if not part:
            continue
        try:
            n = int(part)
            if 0 <= n <= 12:
                xs.append(n)
        except:
            pass
    return xs or DEFAULT_FACTORS

def make_problem(factors: List[int], mode: str) -> Problem:
    '''Build a multiplication or division problem using the chosen factors.'''
    if mode == 'multiply':
        a = rand_item(factors); b = rand_item(factors)
        return Problem(a,b,'×', f'{a} × {b} = ?', a*b)
    if mode == 'divide':
        a = rand_item(factors); b = rand_item(factors)
        dividend = a*b
        divisor = rand_item([a,b])
        answer = dividend // divisor if divisor else 0
        return Problem(a,b,'÷', f'{dividend} ÷ {divisor} = ?', answer, dividend, divisor)
    # mixed: randomly pick
    return make_problem(factors, 'multiply' if random.random()<0.5 else 'divide')

class Game:
    '''Holds the entire game state and rules for moving/answering.'''
    def __init__(self):
        self.size = size_dropdown.value
        self.start = (self.size-1, 0)   # bottom-left
        self.goal  = (0, self.size-1)   # top-right
        self.pos   = self.start
        self.visited = {self.start}
        self.factors = parse_factors(factors_text.value)
        self.mode = mode_dropdown.value
        self.attempts = 0
        self.correct  = 0
        self.clue_chars: List[str] = []  # revealed letters
        self.phrase = phrase_dropdown.value
        self.msg = 'Press an arrow to try the next square.'
        self.problem: Optional[Problem] = None

    def reset(self):
        '''Reset to a fresh game with current settings.'''
        self.__init__()

    def is_adjacent(self, a: Tuple[int,int], b: Tuple[int,int]) -> bool:
        '''Check if b is one step up/down/left/right from a (no diagonals).'''
        return (abs(a[0]-b[0])==1 and a[1]==b[1]) or (abs(a[1]-b[1])==1 and a[0]==b[0])

    def open_problem_for(self, tile: Tuple[int,int]):
        '''Create a new Problem when trying to move to a neighboring tile.'''
        if not self.is_adjacent(self.pos, tile):
            self.msg = 'Pick a square next to your token.'
            return
        self.target = tile
        self.problem = make_problem(self.factors, self.mode)

    def submit(self, user_answer: str) -> bool:
        '''Check the player's answer. If correct, move and award a clue letter.'''
        if not self.problem:
            return False
        self.attempts += 1
        try:
            val = int(user_answer)
        except:
            self.msg = 'Please enter a whole number.'
            return False

        if val == self.problem.answer:
            self.correct += 1
            self.pos = self.target
            self.visited.add(self.pos)
            self.problem = None

            # Reveal one new character from the secret phrase (skip spaces)
            remaining = [ch for ch in self.phrase if (ch != ' ' and ch not in self.clue_chars)]
            if remaining:
                self.clue_chars.append(rand_item(remaining))

            if self.pos == self.goal:
                self.msg = '🎉 You reached the Gym! Can you read the mystery phrase?'
            else:
                self.msg = rand_item([
                    'Great move! Press an arrow to keep exploring.',
                    'Nice! Your partner cheers quietly.',
                    'That fact is yours now. Onward!',
                    'A gentle breeze reveals a new glyph on the path.',
                ])
            return True
        else:
            self.msg = 'Nice try! Tap “Show me a hint” and try again.'
            return False

G = Game()



## 5) Drawing the Board (with Legal‑Move Highlights)

This makes a checkerboard. It **highlights legal moves** (up/down/left/right).  
**Fix included:** START is bottom‑left; GYM is top‑right.


In [ ]:

def draw_board(g: Game, token_img=None, token_emoji='⚡'):
    '''Draw the square board, mark START/GYM, highlight legal moves, and show the token.'''
    n = g.size
    fig, ax = plt.subplots(figsize=(5,5))

    # Create a checkerboard using a simple parity pattern.
    board = np.add.outer(np.arange(n), np.arange(n)) % 2
    ax.imshow(board, interpolation='nearest')
    ax.set_xticks(range(n)); ax.set_yticks(range(n))
    ax.set_xticklabels([]); ax.set_yticklabels([])
    ax.grid(False)

    # Labels — imshow origin is top-left by default.
    # Bottom-left = (x=0, y=n-1), Top-right = (x=n-1, y=0)
    ax.text(0, n-1, 'START', ha='center', va='center', fontsize=12)
    ax.text(n-1, 0, 'GYM', ha='center', va='center', fontsize=12)

    # Current position
    r, c = g.pos

    # Highlight legal neighbors inside board bounds
    for (rr, cc) in [(r-1,c),(r+1,c),(r,c-1),(r,c+1)]:
        if 0 <= rr < n and 0 <= cc < n:
            ax.add_patch(Rectangle((cc-0.5, rr-0.5), 1, 1, fill=False, linewidth=2))

    # Token
    if token_img is not None:
        extent = [c-0.4, c+0.4, r+0.4, r-0.4]  # left, right, bottom, top
        ax.imshow(token_img, extent=extent)
    else:
        ax.text(c, r, token_emoji, ha='center', va='center', fontsize=22)

    plt.show()



## 6) Hints — Number Line, Equal Groups, and Fact Web

These helpers visualize strategies used in class:
- **Number line** — repeated jumps for multiplication or steps for division.
- **Equal groups** — show groups and how many in each.
- **Fact web** — show the related multiplication/division facts.


In [ ]:

def number_line_steps(prob: Problem) -> List[int]:
    '''Return the list of cumulative positions to show on a number line.'''
    if prob.op == '×':
        step = prob.a; times = prob.b
        return [(i+1)*step for i in range(times)]
    else:  # dividend ÷ divisor
        step = prob.divisor if prob.divisor else 1
        target = prob.dividend if prob.dividend else 0
        jumps = max(1, target // max(1, step))
        return [(i+1)*step for i in range(jumps)]

def show_number_line(prob: Problem):
    '''Plot a simple number line with equal jumps.'''
    steps = number_line_steps(prob)
    if not steps:
        display(Markdown('_No steps to show._')); return
    maxv = steps[-1]
    ticks = max(5, min(10, math.ceil(maxv/2)))
    xs = np.linspace(0, maxv, ticks+1)

    fig, ax = plt.subplots(figsize=(5,1.5))
    ax.hlines(0, 0, maxv)  # base line
    for x in xs:
        ax.vlines(x, -0.05, 0.05)
        ax.text(x, -0.15, str(int(round(x))), ha='center', va='top', fontsize=8)
    last = 0
    for s in steps:
        ax.hlines(0, last, s, linewidth=3)
        ax.text(s, 0.1, str(s), ha='center', va='bottom', fontsize=8)
        last = s
    ax.set_ylim(-0.3, 0.3); ax.set_xlim(0, maxv)
    ax.axis('off')
    plt.show()

def show_equal_groups(prob: Problem):
    '''Visualize groups and how many are in each group (no explicit colors).'''
    if prob.op == '×':
        groups, in_each = prob.a, prob.b
    else:
        groups, in_each = (prob.divisor or 1), prob.answer

    cols = min(5, groups)
    rows = math.ceil(groups/cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols*1.6, rows*1.6))
    if not isinstance(axes, np.ndarray):
        axes = np.array([[axes]])
    axes = axes.flatten()
    for g in range(groups):
        ax = axes[g]
        ax.set_title(f'Group {g+1}', fontsize=8)
        side = max(1, math.ceil(math.sqrt(in_each)))
        grid = np.zeros((side, side))
        ax.imshow(grid)
        ax.set_xticks([]); ax.set_yticks([])
        ax.text(0.5, 0.5, f'{in_each}', transform=ax.transAxes, ha='center', va='center', fontsize=10)
    for k in range(groups, len(axes)):
        axes[k].axis('off')
    plt.tight_layout()
    plt.show()

def fact_web(prob: Problem):
    '''Show the family of related facts as simple text.'''
    if prob.op == '×':
        a, b = prob.a, prob.b; p = a*b
        facts = [f'{a} × {b} = {p}', f'{b} × {a} = {p}', f'{p} ÷ {a} = {b}', f'{p} ÷ {b} = {a}']
    else:
        d, v, q = prob.dividend, prob.divisor, prob.answer
        facts = [f'{q} × {v} = {d}', f'{v} × {q} = {d}', f'{d} ÷ {v} = {q}', f'{d} ÷ {q} = {v}']
    display(Markdown('**Fact Web**  \n' + ' · '.join(facts)))



## 7) Game UI — Buttons, Hints, and Live Status

Left side:
- **How to play** box
- Arrow buttons
- Answer box, Submit/Cancel/New Board, and **Show me a hint**

Right side:
- The board (with legal moves outlined)


In [ ]:

# --- Outputs that change while playing ---
board_out = W.Output()
hints_out = W.Output()

# --- Kid‑friendly help box ---
help_html = '''
<div style="font-family:system-ui; line-height:1.35">
  <h3 style="margin:0 0 .25rem 0">How to play</h3>
  <ol style="margin:.25rem 0 .5rem 1.25rem; padding:0">
    <li><b>Press an arrow</b> to try moving to the next square.</li>
    <li><b>Answer the math puzzle</b> to make the move.</li>
    <li>Each correct answer <b>reveals a letter</b> in the mystery phrase.</li>
    <li>Reach the <b>GYM</b> to finish the adventure!</li>
  </ol>
  <p style="margin:0"><i>Need help?</i> Tap <b>Show me a hint</b> for number line, equal groups, and fact-web ideas.</p>
</div>
'''
help_box = W.HTML(help_html)

# --- Controls (larger buttons, clear labels) ---
ans = W.Text(placeholder='Type your answer here', layout=W.Layout(width='260px'))

submit_btn = W.Button(description='Submit ✅', button_style='primary', layout=W.Layout(width='120px'))
cancel_btn  = W.Button(description='Cancel', layout=W.Layout(width='90px'))
refresh_btn = W.Button(description='New Board', layout=W.Layout(width='120px'))
hint_btn    = W.Button(description='Show me a hint 💡', layout=W.Layout(width='150px'))

up_btn    = W.Button(description='↑ Up',    layout=W.Layout(width='120px', height='36px'))
down_btn  = W.Button(description='↓ Down',  layout=W.Layout(width='120px', height='36px'))
left_btn  = W.Button(description='← Left',  layout=W.Layout(width='120px', height='36px'))
right_btn = W.Button(description='→ Right', layout=W.Layout(width='120px', height='36px'))

status = W.HTML('')
story = W.HTML('')
clue_out = W.HTML('')

def token_image_or_none():
    '''Return a Pillow image or None based on the current token selection.'''
    if token_type.value == 'Image' and token_image.value != '(none)':
        return load_token_image(Path(ASSETS_DIR) / token_image.value)
    return None

def refresh_all():
    '''Redraw the board and update text panels.'''
    board_out.clear_output(wait=True)
    with board_out:
        draw_board(G, token_img=token_image_or_none(), token_emoji=token_emoji.value)

    status.value = f'<b>Correct:</b> {G.correct} · <b>Attempts:</b> {G.attempts}<br/><span>{G.msg}</span>'

    # Reveal the mystery phrase with dots for letters not yet found
    revealed = ''.join([ch if ch in G.clue_chars or ch==' ' else '•' for ch in G.phrase])
    clue_out.value = f'<b>Mystery Phrase:</b> <span style=\"font-family:monospace\">{revealed}</span>'

    # Show a story line based on progress
    idx = min(len(G.visited)-1, len(STORY_BEATS)-1)
    story.value = f'<i>{STORY_BEATS[idx]}</i>'

def try_move(dr, dc):
    '''Attempt to move in a direction and open a problem if legal.'''
    r, c = G.pos
    nr, nc = clamp(r+dr, 0, G.size-1), clamp(c+dc, 0, G.size-1)
    if (nr, nc) == G.pos:
        G.msg = 'Edge of the board. Try another direction.'
        refresh_all(); return
    if not G.is_adjacent(G.pos, (nr, nc)):
        G.msg = 'Pick a neighboring square.'
        refresh_all(); return
    G.open_problem_for((nr, nc))
    if G.problem:
        with hints_out:
            hints_out.clear_output(wait=True)
            display(Markdown(f'### Solve to move: **{G.problem.prompt}**'))
        ans.value = ''
        ans.focus()

def on_submit(_):
    '''Check the user\'s answer and update the UI.'''
    ok = G.submit(ans.value.strip())
    refresh_all()
    if ok:
        hints_out.clear_output(wait=True)

def on_cancel(_):
    '''Close the current problem without moving.'''
    G.problem = None
    G.msg = 'Okay — press an arrow when you\'re ready.'
    hints_out.clear_output(wait=True)
    refresh_all()

def on_new_board(_):
    '''Create a fresh game using the current settings.'''
    G.size = size_dropdown.value
    G.factors = parse_factors(factors_text.value)
    G.mode = mode_dropdown.value
    G.phrase = phrase_dropdown.value
    G.reset()
    refresh_all()

def on_hint(_):
    '''Show hint visuals for the current problem (or prompt to start).'''
    if G.problem:
        with hints_out:
            hints_out.clear_output(wait=True)
            display(Markdown(f'### Try a helper for **{G.problem.prompt}**'))
            if show_numline.value:
                display(Markdown('**Number Line**')); show_number_line(G.problem)
            if show_groups.value:
                display(Markdown('**Equal Groups**')); show_equal_groups(G.problem)
            if show_factweb.value:
                fact_web(G.problem)
    else:
        G.msg = 'Press an arrow first to get a puzzle, then tap “Show me a hint”.'
        refresh_all()

# Wire up button actions
submit_btn.on_click(on_submit)
cancel_btn.on_click(on_cancel)
refresh_btn.on_click(on_new_board)
hint_btn.on_click(on_hint)

up_btn.on_click(lambda _: try_move(-1, 0))
down_btn.on_click(lambda _: try_move(1, 0))
left_btn.on_click(lambda _: try_move(0, -1))
right_btn.on_click(lambda _: try_move(0, 1))

# Build the left side (controls) and right side (board) layout
controls = W.VBox([
    W.HTML('<b>Play</b>'),
    help_box,
    W.HBox([up_btn, down_btn, left_btn, right_btn]),
    W.HBox([ans, submit_btn, cancel_btn, refresh_btn, hint_btn]),
    status, story, clue_out,
])

# Display the app
display(W.HBox([W.VBox([controls, hints_out], layout=W.Layout(width='55%')), board_out]))

# Initialize the first board
on_new_board(None)



## 8) Tips for Grown‑Ups

- Keep sessions short (10–15 minutes). Let your child steer the pace.
- Toggle hints on at first; hide them gradually as facts become automatic.
- Adjust **Factors** as mastery grows (add 3,4,6,7,8,9).
- Try **Division** or **Mixed** once multiplication is comfortable.
- Celebrate finishing the **mystery phrase** — not speed.
